In [ ]:
#SCALE FACTOR MENOR: MÁS ROSTROS FALSOS
#SCALE FACTOR MAYOR: MENOS ROSTROS FALSOS DETECTADOS
#DEPENDE DE LA COMPLEJIDAD DE LA IMAGEN
#HAY QUE ENCONTRAR UN VALOR INTERMEDIO

In [ ]:
!pip install --ignore-installed blinker




  Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)


In [ ]:
!pip install opencv-python opencv-python-headless opencv-contrib-python-headless deepface



  Using cached deepface-0.0.92-py3-none-any.whl (105 kB)
  Using cached flask-3.0.3-py3-none-any.whl (101 kB)
  Using cached mtcnn-0.1.1-py3-none-any.whl (2.3 MB)
  Using cached retina_face-0.0.17-py3-none-any.whl (25 kB)


CLASIFICACIÓN DE IMÁGENES SEGÚN NÚMERO DE ROSTROS Y EMOCIÓN PREDOMINANTE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Subdirectorios
directorio_raiz = '/content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset'
#Archivo csv destino
csv_path = '/content/drive/My Drive/TFG PABLO VILLALBA BRISA/Archivos/insta_posts.csv'

Mounted at /content/drive


V2:OPTIMIZACIÓN DE RECURSOS

In [ ]:
import os
import pandas as pd
from deepface import DeepFace
from mtcnn import MTCNN
import cv2
import tempfile
import gc

#Carga detector de rostros MTCNN
detector_rostros = MTCNN()

#Emociones
emociones = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

#Función extraer el shortcode de img
def extraer_shortcode(nombre_imagen):
    if nombre_imagen.endswith("UTC.jpg"):
        return nombre_imagen.replace(".jpg", "")
    elif nombre_imagen.endswith("UTC_1.jpg"):
        return nombre_imagen.replace("_1.jpg", "")
    return None

#Leer csv
df = pd.read_csv(csv_path)

resultados = []

#Tamaño del lote
batch_size = 10
image_paths = []
shortcodes = []

for root, _, files in os.walk(directorio_raiz):
    for nombre_imagen in files:
        if nombre_imagen.endswith("UTC.jpg") or nombre_imagen.endswith("UTC_1.jpg"):
            #Extraer shortcode
            shortcode = extraer_shortcode(nombre_imagen)
            if not shortcode:
                continue

            #Ruta imagen
            image_path = os.path.join(root, nombre_imagen)
            image_paths.append(image_path)
            shortcodes.append(shortcode)

            #Si tamaño lote alcanza límite o es la última imagen
            if len(image_paths) == batch_size:
                #Procesar lote de imgs
                for img_path, sc in zip(image_paths, shortcodes):
                    #Diccionario para contar las emociones para la img específica
                    contador_emociones = {emocion: 0 for emocion in emociones}

                    #Leer img con OpenCV
                    imagen = cv2.imread(img_path)
                    #Pasar a RGB (MTCNN utiliza formato RGB)
                    imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
                    #Detectar rostros en img
                    rostros = detector_rostros.detect_faces(imagen_rgb)

                    #Print número de rostros detectados
                    print("Número de rostros en", img_path, ":", len(rostros))

                    #Analizar emociones en rostros detectados
                    for rostro in rostros:
                        x, y, w, h = rostro['box'] if 'box' in rostro else (0, 0, 0, 0)
                        #Recortar el área de la cara para detectar
                        roi_gray = imagen_rgb[y:y+h, x:x+w]
                        #Guardar la imagen recortada temporalmente
                        with tempfile.NamedTemporaryFile(suffix='.jpg') as temp_file:
                            cv2.imwrite(temp_file.name, roi_gray)
                            #Predección de emoción sobre el recorte con DeepFace
                            emotion_prediction = DeepFace.analyze(img_path=temp_file.name, actions=['emotion'], enforce_detection=False)
                            #Verificar que emoción detectada está en la lista de emociones
                            if emotion_prediction and 'dominant_emotion' in emotion_prediction[0] and emotion_prediction[0]['dominant_emotion'] in emociones:
                                #Incrementar el contador de esa emoción que se ha detectado
                                contador_emociones[emotion_prediction[0]['dominant_emotion']] += 1

                    #Determinar la emoción predominante img
                    emocion_predominante = max(contador_emociones, key=contador_emociones.get)
                    print("Emoción predominante en", img_path, ":", emocion_predominante)

                    #Añadir res
                    resultados.append((sc, len(rostros), emocion_predominante))

                #Limpiar listas
                image_paths = []
                shortcodes = []

                #Liberar memoria
                gc.collect()

#Si hay imágenes restantes en listas
if image_paths:
    for img_path, sc in zip(image_paths, shortcodes):
        contador_emociones = {emocion: 0 for emocion in emociones}

        imagen = cv2.imread(img_path)
        imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
        rostros = detector_rostros.detect_faces(imagen_rgb)

        print("Número de rostros en", img_path, ":", len(rostros))

        for rostro in rostros:
            x, y, w, h = rostro['box'] if 'box' in rostro else (0, 0, 0, 0)
            roi_gray = imagen_rgb[y:y+h, x:x+w]
            with tempfile.NamedTemporaryFile(suffix='.jpg') as temp_file:
                cv2.imwrite(temp_file.name, roi_gray)
                emotion_prediction = DeepFace.analyze(img_path=temp_file.name, actions=['emotion'], enforce_detection=False)

                if emotion_prediction and 'dominant_emotion' in emotion_prediction[0] and emotion_prediction[0]['dominant_emotion'] in emociones:
                    contador_emociones[emotion_prediction[0]['dominant_emotion']] += 1

        emocion_predominante = max(contador_emociones, key=contador_emociones.get)
        print("Emoción predominante en", img_path, ":", emocion_predominante)

        resultados.append((sc, len(rostros), emocion_predominante))

    gc.collect()

df_resultados = pd.DataFrame(resultados, columns=["post", "Rostros", "Emoción"])

df_actualizado = pd.merge(df, df_resultados, on="post", how="left")

df_actualizado.to_csv(csv_path, index=False)



Se han truncado las últimas 5000 líneas del flujo de salida.
1/1 [==============================] - 0s 30ms/step
Número de rostros en /content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset/abdullahshelbayhh/2023-12-28_20-56-38_UTC.jpg : 2
Emoción predominante en /content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset/abdullahshelbayhh/2023-12-28_20-56-38_UTC.jpg : angry
1/1 [==============================] - 0s 29ms/step
Número de rostros en /content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset/albertobarroso96/2023-04-27_19-06-30_UTC.jpg : 1
Emoción predominante en /content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset/albertobarroso96/2023-04-27_19-06-30_UTC.jpg : neutral
1/1 [==============================] - 0s 30ms/step
Número de rostros en /content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset/albertobarroso96/2023-05-26_14-02-54_UTC_1.jpg : 3
Emoción predominante en /content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset/albertobarroso96/2023-0

Versión para almacenar predicciones en un CSV independiente:

In [ ]:
import os
import pandas as pd
from deepface import DeepFace
from mtcnn import MTCNN
import cv2
import tempfile
import gc

detector_rostros = MTCNN()

emociones = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

def extraer_shortcode(nombre_imagen):
    if nombre_imagen.endswith("UTC.jpg"):
        return nombre_imagen.replace("UTC.jpg", "")
    elif nombre_imagen.endswith("UTC_1.jpg"):
        return nombre_imagen.replace("UTC_1.jpg", "")
    return None

directorio_raiz = '/content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset'
nuevo_csv_path = '/content/drive/My Drive/TFG PABLO VILLALBA BRISA/Archivos/face_emotions_prediction.csv'

predicciones = {'username': [], 'post': [], 'rostros': [], 'emocion': []}

batch_size = 10
image_paths = []
shortcodes = []
usernames = []

for subdir, _, files in os.walk(directorio_raiz):
    for nombre_imagen in files:
        if nombre_imagen.endswith("UTC.jpg") or nombre_imagen.endswith("UTC_1.jpg"):
            shortcode = extraer_shortcode(nombre_imagen)
            if not shortcode:
                continue

            image_path = os.path.join(subdir, nombre_imagen)
            image_paths.append(image_path)
            shortcodes.append(shortcode)

            usuario = os.path.basename(subdir)
            usernames.append(usuario)

            if len(image_paths) == batch_size:
                for img_path, sc, usuario in zip(image_paths, shortcodes, usernames):
                    contador_emociones = {emocion: 0 for emocion in emociones}

                    imagen = cv2.imread(img_path)
                    imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
                    rostros = detector_rostros.detect_faces(imagen_rgb)

                    print("Número de rostros en", img_path, ":", len(rostros))

                    for rostro in rostros:
                        x, y, w, h = rostro['box'] if 'box' in rostro else (0, 0, 0, 0)
                        roi_gray = imagen_rgb[y:y+h, x:x+w]
                        with tempfile.NamedTemporaryFile(suffix='.jpg') as temp_file:
                            cv2.imwrite(temp_file.name, roi_gray)
                            emotion_prediction = DeepFace.analyze(img_path=temp_file.name, actions=['emotion'], enforce_detection=False)
                            if emotion_prediction and 'dominant_emotion' in emotion_prediction[0] and emotion_prediction[0]['dominant_emotion'] in emociones:
                                contador_emociones[emotion_prediction[0]['dominant_emotion']] += 1

                    emocion_predominante = max(contador_emociones, key=contador_emociones.get)
                    print("Emoción predominante en", img_path, ":", emocion_predominante)

                    predicciones['username'].append(usuario)
                    predicciones['post'].append(sc)
                    predicciones['rostros'].append(len(rostros))
                    predicciones['emocion'].append(emocion_predominante)

                image_paths = []
                shortcodes = []
                usernames = []

                gc.collect()

if image_paths:
    for img_path, sc, usuario in zip(image_paths, shortcodes, usernames):
        contador_emociones = {emocion: 0 for emocion in emociones}

        imagen = cv2.imread(img_path)
        imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
        rostros = detector_rostros.detect_faces(imagen_rgb)

        print("Número de rostros en", img_path, ":", len(rostros))

        for rostro in rostros:
            x, y, w, h = rostro['box'] if 'box' in rostro else (0, 0, 0, 0)
            roi_gray = imagen_rgb[y:y+h, x:x+w]
            with tempfile.NamedTemporaryFile(suffix='.jpg') as temp_file:
                cv2.imwrite(temp_file.name, roi_gray)
                emotion_prediction = DeepFace.analyze(img_path=temp_file.name, actions=['emotion'], enforce_detection=False)
                if emotion_prediction and 'dominant_emotion' in emotion_prediction[0] and emotion_prediction[0]['dominant_emotion'] in emociones:
                    contador_emociones[emotion_prediction[0]['dominant_emotion']] += 1

        emocion_predominante = max(contador_emociones, key=contador_emociones.get)
        print("Emoción predominante en", img_path, ":", emocion_predominante)

        predicciones['username'].append(usuario)
        predicciones['post'].append(sc)
        predicciones['rostros'].append(len(rostros))
        predicciones['emocion'].append(emocion_predominante)

    gc.collect()

df_predicciones = pd.DataFrame(predicciones)

df_predicciones.to_csv(nuevo_csv_path, index=False)


1/1 [==============================] - 1s 515ms/step
Número de rostros en /content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset/alexdeminaur/2023-01-02_12-50-24_UTC_1.jpg : 1
24-06-27 11:15:19 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 15.5MB/s]


Se han truncado las últimas 5000 líneas del flujo de salida.
1/1 [==============================] - 0s 28ms/step
Número de rostros en /content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset/abdullahshelbayhh/2023-12-28_20-56-38_UTC.jpg : 2
Emoción predominante en /content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset/abdullahshelbayhh/2023-12-28_20-56-38_UTC.jpg : angry
1/1 [==============================] - 0s 30ms/step
Número de rostros en /content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset/albertobarroso96/2023-04-27_19-06-30_UTC.jpg : 1
Emoción predominante en /content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset/albertobarroso96/2023-04-27_19-06-30_UTC.jpg : neutral
1/1 [==============================] - 0s 32ms/step
Número de rostros en /content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset/albertobarroso96/2023-05-26_14-02-54_UTC_1.jpg : 3
Emoción predominante en /content/drive/My Drive/TFG PABLO VILLALBA BRISA/TFG_Dataset/albertobarroso96/2023-0